1. Оцените возможности Sculpting, в <b>Wizard -> Demo -> Sculpting</b>.

        before demo
!['before demo'](demo_before.png)


        after demo
!['after demo'](demo_after.png)

    Поработайте в PyMOL с записью 1LMP банка PDB.
    
2. Используюя <i>select within</i> найдите водородные связи которыми белок связывает лиганд.

    Команды для скрипта, использованные в PyMOL.

fetch 1lmp
extract water, resn HOH
delete water
select ligand, resn NAG+NDG
util.cbaw
select prot, not ligand
color lightpink, prot
set dash_width , 1
select cont, byres prot within 3.5 of ligand
show lines, cont
color lightblue, cont
distance hbond, cont, ligand , 3.5 , 2
set dash_color, yellow
zoom cont

        1LMP and ligand
!['1LMP and ligand'](1LMP.png)

        1LMP and ligand whith bonds
!['1LMP and ligand whith bonds'](1LMP_ligand.png)

        1LMP and ligand whith bonds closer
!['1LMP and ligand whith bonds closer'](1LMP_ligand_close.png)

3. Средствами Tcl/Tk интерфейса <b>Wizard->Mutagenesis</b> проведите мутацию в белке, которая по вашему мнению должна привести к потере связывания с лигандом.

Точно сложно сказать, какая мутация точно приведет к потере связывания с лигандом. Но я выбрала ASP под номером 101 и заменила его на TRP, хотя не знаю, действительно ли такая замена приведет к потеря связи с лигандом. Замена производилась непосредственно в программе, без кода. Белок сохранён как <b>1lmp_mut.pdb</b>.

4. Используя команды <i>mset, mview, super, translate</i> создайте анимационный ролик mpeg где происходит совмещение белков и показывается место мутации.

Скрипт для создания анимации.

![SegmentLocal](Rastvorova_task2_movie.gif "segment")

5. Приседените флуорусцентную метку TAMRA к белку через сложноэфирную связь. Используйте команды <i>fuse и torsion</i>.

Сначала посмотрим на белок и найдём аминокислоты, к которым может присоединиться Tamra через эфирную связь. Такие аминокислоты должны содержать -OH группу, так как Tamra имеет карбоксильную. Найдём все серины и выберем тот, который расположен на поврхности. Я выбрала серин под номером 122.

![122](122.png)

Теперь посмотрим на флуорисцентную метку Tamra. И удалим группу -OH, которая будет участвовать в реакции этерификации. У белка атомы водорода не указаны, поэтому у него ничего не удаляем.

![tamra](tamra.png)

Выберем атомы, которые будут участвовать в реакции и соеденим их.
С помощью <i>torsion</i> будем изменять градус, чтобы подобрать наиболее оптимальное положение метки.

![tamra_n_1lmp_1](tamra_n_1lmp_1.png)
![tamra_n_1lmp_2](tamra_n_1lmp_2.png)

6. Напишите скрипт для построения поли-аланиновой альфа-спирали длинной 100 аминкислот.

Я выбрала триптофан.

![trp1](trp.png)
![trp2](trp2.png)